In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

In [2]:
training_data = pd.read_csv("/Users/kushalgupta/Downloads/twitter_train.csv")
testing_data = pd.read_csv("/Users/kushalgupta/Downloads/twitter_test.csv")

In [3]:
training_data = training_data[["airline_sentiment", "text"]]
testing_data = testing_data[["text"]]

In [4]:
training_data.head()

,airline_sentiment,text
0,negative,"@SouthwestAir I am scheduled for the morning, ..."
1,positive,@SouthwestAir seeing your workers time in and ...
2,positive,@united Flew ORD to Miami and back and had gr...
3,negative,@SouthwestAir @dultch97 that's horse radish 😤🐴
4,negative,@united so our flight into ORD was delayed bec...


In [5]:
data_train = np.array(training_data.text)
target = np.array(training_data.airline_sentiment)
data_test = np.array(testing_data.text)

In [6]:
print(data_train.shape)
print(target.shape)
print(data_test.shape)

(10980,)
(10980,)
(3660,)


In [7]:
tok_train = list()
for i in range(len(data_train)):
    tok_train.append(word_tokenize(data_train[i]))

tok_test = list()
for j in range(len(data_test)):
    tok_test.append(word_tokenize(data_test[j]))

In [8]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [9]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    

In [10]:
from nltk import pos_tag

In [11]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
import string
punctuations = list(string.punctuation)
stop = stop + punctuations

In [12]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stop:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [13]:
tok_train = [clean_review(i) for i in tok_train]
tok_test = [clean_review(i) for i in tok_test]

In [14]:
X = list()
for doc in tok_train:
    X.append(" ".join(doc))
    
final_test = list()
for doct in tok_test:
    final_test.append(" ".join(doct))
    

In [17]:
x_train,x_test,y_train,y_test = train_test_split(X,target)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
#count_vec = CountVectorizer(max_features = 2000)
count_vec = CountVectorizer(max_features = 2000, ngram_range = (1,2), max_df = 0.8)


x_train_features = count_vec.fit_transform(X)
x_train_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [26]:
x_test_features = count_vec.transform(final_test)

In [27]:
from sklearn.linear_model import LogisticRegression

In [28]:
clf = LogisticRegression()
clf.fit(x_train_features,target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [29]:
y_pred = clf.predict(x_test_features)

In [30]:
np.savetxt("twitter_logistic.csv", y_pred,delimiter = ',',fmt = '%s')

In [24]:
clf.score(x_test_features,y_test)

0.7868852459016393

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [ ]:
clf = SVC()

In [ ]:
grid = {'C' : [1e2, 1e3, 5e3, 1e4, 5e4, 1e5], 'gamma' : [1e-3, 5e-3, 1e-4, 5e-4]}
abc =  GridSearchCV(clf, grid)
abc.fit(x_train_features,target)

abc.best_estimator_

In [ ]:
clf = SVC(C = 1000, gamma = 0.0001)
clf.fit(x_train_features,target)

In [ ]:
y_svm = clf.predict(x_test_features)

In [ ]:
np.savetxt("twitter_svm.csv", y_svm,delimiter = ',',fmt = '%s')

In [ ]:
clf.score(x_test_features,y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(x_train_features, y_train)

In [ ]:
rfc.score(x_test_features, y_test)